In [27]:
import torch
from torchvision.models import resnet18, ResNet18_Weights # torchvision에서 학습된 ResNet18 모델을 불러옴. 
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

In [28]:
# 입력(input) 데이터를 모델의 각 층(layer)에 통과시켜 예측값(prediction)을 생성
prediction = model(data) # 순전파 단계(forward pass)를 수행하여 예측값(prediction)을 생성

In [29]:
# 모델의 예측값과 그에 해당하는 정답(label)을 사용하여 오차(error, 손실(loss))를 계산
# 오차 텐서(error tensor)에 .backward() 메서드를 호출하여 모델의 매개변수(parameter)에 대한 손실(loss)의 그래디언트(gradient)를 계산
loss = (prediction - labels).sum()
loss.backward() # 역전파 단계(backward pass)

In [30]:
# 옵티마이저(optimizer)에 모델의 모든 매개변수를 등록. 이 예제에서는 학습률(learning rate) 0.1과 모멘텀(momentum) 0.9를 갖는 SGD입니다. 
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [31]:
optim.step() # 옵티마이저(optimizer)의 step() 메서드를 호출하여 모델의 매개변수를 업데이트합니다.

# Autograd에서 미분

In [32]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)


In [33]:
# 이제 a와 b로부터 새로운 텐서 Q를 만듦
Q = 3*a**3 - b**2

In [34]:
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad) # 변화도는 a.grad와 b.grad에 저장됩니다.
a.grad

tensor([36., 81.])

In [35]:
# 수집된 변화도가 올바른지 확인합니다.
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


# DAG에서 제외하기

In [36]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")


Does `a` require gradients?: False
Does `b` require gradients?: True


In [40]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT) # 모델을 불러옴

# 신경망의 모든 매개변수를 고정합니다
for param in model.parameters():
    param.requires_grad = False # 역전파를 통해 매개변수의 변화도가 계산되지 않도록 requires_grad를 False로 설정
                                

model.fc = nn.Linear(512, 10) # 새로운 분류기로 동작할 새로운 선형 계층으로 간단히 대체
                              # 마지막 출력층을 관례적으로 fc라고 지어 둚. 그래서 모델의 마지막 계층을 새로 정의한 선형 계층으로 대체

# 분류기만 최적화합니다.
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)